<a href="https://colab.research.google.com/github/rahiakela/practical-machine-learning-with-tensorflow/blob/week-7/assignment_7_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation with an RNN

## Setup

### Import TensorFlow and other libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

tf.random.set_seed(1)

TensorFlow 2.x selected.


### Download the Shakespeare dataset

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read the data

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [4]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [0]:
# Create mapping from unique characters to indices
char2idx = {u: i for i, u in  enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print('{')
for char, _ in zip(char2idx, range(20)):
  print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('   ...\n') 

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
   ...



In [7]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## The prediction task

### Create training examples and targets

In [0]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [11]:
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Create training batches

In [15]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

* tf.keras.layers.Embedding (dim=256)
* tf.keras.layers.LSTM (units=1024)
* tf.keras.layers.Dense

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)                                
  ])

  return model

In [0]:
model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

### Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Train the model

#### Attach an optimizer, and a loss function

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print('Prediction shape: ', example_batch_predictions.shape, ' # (batch_size, sequence_length, vocab_size)')
print('scalar_loss:      ', example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174328


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

#### 6) Build the given model and train it for 10 epochs. What is the range of average loss of the trained model?

In [0]:
EPOCHS=10

In [0]:
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

#### 7) Now train the same model more 30 epochs. What is the range of average loss of the trained model?

In [0]:
# history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback])

#### 8) It looks like the model is not able to generate meaningful text. Possible improvements can be:

In [0]:
def build_model_2(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.models.Sequential([
     tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
     tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dense(vocab_size)                                 
  ])

  return model

In [0]:
model_2 = build_model_2(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [0]:
model_2.compile(optimizer='adam', loss=loss)

In [30]:
history = model_2.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 31s 181ms/step - loss: 2.5867
Epoch 2/10
172/172 [==============================] - 29s 170ms/step - loss: 1.8728
Epoch 3/10
172/172 [==============================] - 30s 172ms/step - loss: 1.6280
Epoch 4/10
172/172 [==============================] - 29s 171ms/step - loss: 1.4984
Epoch 5/10
172/172 [==============================] - 30s 172ms/step - loss: 1.4199
Epoch 6/10
172/172 [==============================] - 30s 172ms/step - loss: 1.3632
Epoch 7/10
172/172 [==============================] - 30s 172ms/step - loss: 1.3199
Epoch 8/10
172/172 [==============================] - 30s 172ms/step - loss: 1.2809
Epoch 9/10
172/172 [==============================] - 30s 172ms/step - loss: 1.2456
Epoch 10/10
172/172 [==============================] - 29s 171ms/step - loss: 1.2112


In [31]:
model_3 = build_model(vocab_size=len(vocab), embedding_dim=512, rnn_units=rnn_units, batch_size=BATCH_SIZE)
model_3.compile(optimizer='adam', loss=loss)
history = model_3.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 34s 200ms/step - loss: 2.5087
Epoch 2/10
172/172 [==============================] - 33s 191ms/step - loss: 1.8298
Epoch 3/10
172/172 [==============================] - 33s 192ms/step - loss: 1.5994
Epoch 4/10
172/172 [==============================] - 33s 193ms/step - loss: 1.4798
Epoch 5/10
172/172 [==============================] - 33s 192ms/step - loss: 1.4064
Epoch 6/10
172/172 [==============================] - 33s 191ms/step - loss: 1.3548
Epoch 7/10
172/172 [==============================] - 33s 192ms/step - loss: 1.3128
Epoch 8/10
172/172 [==============================] - 33s 192ms/step - loss: 1.2755
Epoch 9/10
172/172 [==============================] - 33s 192ms/step - loss: 1.2422
Epoch 10/10
172/172 [==============================] - 33s 192ms/step - loss: 1.2086


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model_2 = build_model_2(vocab_size, embedding_dim, rnn_units, batch_size=1)

model_2.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model_2.build(tf.TensorShape([1, None]))

In [0]:
model_2.summary()

### The prediction loop

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model_2, start_string=u"ROMEO: "))